In [112]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_42857/3857342807.py:17: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('flights.csv')


In [113]:
flights = flights.merge(airports, how='right', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE')

добавляю колонки час и минуты и формирую соответствующую колонку DATE
выбираю период по 30 сентября
не нашел коды аэропортов, в октябре они цифрами прописаны, поэтому теряю месяц (октябрь)

In [114]:
flights['HOUR'] = flights['SCHEDULED_DEPARTURE']//100
flights['MINUTE'] = flights['SCHEDULED_DEPARTURE']%100
flights['DATE'] = pd.to_datetime(flights[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']])
flights = flights.loc[flights['DATE'].between('2015-01-01', '2015-09-30')]


формирую колонку C_TIME с количеством рейсов в d каждый момент DATE

In [115]:
c_time = flights.groupby('DATE').count().reset_index()
c_time['C_TIME'] = c_time['MONTH']
c_time = c_time[['DATE', 'C_TIME']]
flights = flights.merge(c_time, how='left', left_on='DATE', right_on='DATE')

теперь выбираю данные для обучения, все они не содержат времени, связанного с задержкой

In [116]:
my_flight = flights[['DAY_OF_WEEK', 'HOUR', 'MINUTE',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_TIME',
       'DATE', 'ARRIVAL_DELAY',
       'C_TIME', 'AIRLINE', 'FLIGHT_NUMBER', 'DISTANCE', 'TAIL_NUMBER' ]]

небольшой пример

In [117]:
ATL = my_flight.query("ORIGIN_AIRPORT == 'MEM'")
ATL_IAD = ATL.query("DESTINATION_AIRPORT =='CVG'")

In [118]:
ATL_IAD.set_index('DATE', inplace=True)
ATL_IAD.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], axis=1, inplace=True)

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_42857/1282734093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ATL_IAD.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], axis=1, inplace=True)


In [119]:
le = LabelEncoder()
ATL_IAD.AIRLINE = le.fit_transform(ATL_IAD['AIRLINE'])
ATL_IAD.TAIL_NUMBER = le.fit_transform(ATL_IAD['TAIL_NUMBER'])

ATL_IAD.dropna(inplace=True)

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_42857/1462679028.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ATL_IAD.AIRLINE = le.fit_transform(ATL_IAD['AIRLINE'])
/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_42857/1462679028.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ATL_IAD.TAIL_NUMBER = le.fit_transform(ATL_IAD['TAIL_NUMBER'])
/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_42857/1462679028.py:5: SettingWithCopyWarning: 
A value is trying to be set on 

In [120]:
X_train,X_test, y_train, y_test = train_test_split(ATL_IAD.drop('ARRIVAL_DELAY',axis=1),
                                                  ATL_IAD.ARRIVAL_DELAY,
                                                  shuffle = False,
                                                  test_size = 0.25)

Ошибка не такая значительная, как в примере baseline

In [121]:
lr = LinearRegression()
#sc = StandardScaler()
lr.fit(X_train, y_train)
print('Ошибка на обучающей выборке', (mean_squared_error(y_train, lr.predict(X_train)))**0.5)
pred = lr.predict(X_test)
print('Ошибка на тесте', (mean_squared_error(y_test, pred))**0.5)

Ошибка на обучающей выборке 31.41573069700264
Ошибка на тесте 60.05882379790696


In [122]:
rf = RandomForestRegressor(n_estimators=10, random_state=0)
rf.fit(X_train, y_train)
print('Ошибка на обучающей выборке', (mean_squared_error(y_train, rf.predict(X_train)))**0.5)
pred = rf.predict(X_test)
print('Ошибка на тесте', (mean_squared_error(y_test, pred))**0.5)

Ошибка на обучающей выборке 17.53323374949087
Ошибка на тесте 54.1085852707313
